In [1]:

import sys

sys.path.append("..")
from data.database import SessionLocal
from data.data_models import Products

from tqdm import tqdm

In [2]:

import os
import requests
from openai import OpenAI
from sqlalchemy.orm import Session
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env")

client = OpenAI()
def generate_and_save_image(prompt, filename):
    # Check if the image already exists
    if not os.path.exists(filename):
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )

        image_url = response.data[0].url
        image_response = requests.get(image_url)
        
        # Ensure the directory exists
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        
        with open(filename, 'wb') as f:
            f.write(image_response.content)
    else:
        print(f"Image already exists: {filename}")

def process_restaurants(db: Session):
    products = db.query(Products).all()
    print("Generating restaurant images...")
    print(products)
    for product in tqdm(products):
        try:
            prompt = (
                f"Generate a highly realistic, high-resolution thumbnail image of a {product.name} by {product.brand} with quantity {product.quantity} "
                f"The product should be displayed on a clean, white background with soft, natural shadows to create a realistic depth effect. "
                f"Focus on capturing intricate details, textures, and accurate branding elements of the product. "
                f"Ensure the logo and brand markings are clearly visible and correctly represented. "
                f"The product must be centered and evenly lit to ensure a professional and appealing appearance, suitable for an online store."
            )
            filename = os.path.join('../../../frontend/src/assets/', product.image)
            generate_and_save_image(prompt, filename)
        except Exception as e:
            print(e)


# Get the database session
db = SessionLocal()

try:
    # Process restaurants and foods
    process_restaurants(db)
except Exception as e:
    print(e)

finally:
    # Close the session
    db.close()

Generating restaurant images...
[<data.data_models.Products object at 0x000001ADEC33D310>, <data.data_models.Products object at 0x000001ADEC33D2B0>, <data.data_models.Products object at 0x000001ADEC33D340>, <data.data_models.Products object at 0x000001ADEC33D3A0>, <data.data_models.Products object at 0x000001ADEC33D3D0>, <data.data_models.Products object at 0x000001ADEC33D460>, <data.data_models.Products object at 0x000001ADEC33D4F0>, <data.data_models.Products object at 0x000001ADEC33D580>, <data.data_models.Products object at 0x000001ADEC33D610>, <data.data_models.Products object at 0x000001ADEC33D6A0>, <data.data_models.Products object at 0x000001ADEC33D730>, <data.data_models.Products object at 0x000001ADEC33D8B0>, <data.data_models.Products object at 0x000001ADEC33D940>, <data.data_models.Products object at 0x000001ADEC33D9D0>, <data.data_models.Products object at 0x000001ADEC33DA60>, <data.data_models.Products object at 0x000001ADEC33DAF0>, <data.data_models.Products object at 0x

100%|██████████| 32/32 [10:48<00:00, 20.28s/it]
